### Heroes Of Pymoli Data Analysis
* Of the 576 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  

* Total purchase value for players aged 20-24 is the highest at about $1,114 , meaning they spend the most money.

* Oathbreaker, Last Hope of the Breaking Storm was the most profitable item with $50.8 in total sales. 
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file_to_load = "HeroesOfPymoli/Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# make a copy to do formatting so as not to affect original df
purchase_data_formatted = purchase_data.copy()

# formatting function
def format_currency(x):
    ''' takes a float value and formats it to two decimals '''
    return "${:.2f}".format((x))
    
purchase_data_formatted["Price"] = purchase_data_formatted["Price"].apply(format_currency)

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

## Player Count

* Display the total number of players


In [4]:
total_players = purchase_data["SN"].nunique()
df_total_players = pd.DataFrame({
    "Total Players": [total_players]
})

df_total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, total number of purchases, total revenue


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
total_unique_items = purchase_data["Item ID"].nunique()
average_price = purchase_data["Price"].mean()
total_purchases = purchase_data["Purchase ID"].nunique()
total_revenue = purchase_data["Price"].sum()

df_purchasing_analysis = pd.DataFrame({
    "Number of Unique Items": [total_unique_items],
    "Average Price": [format_currency(average_price)],
    "Number of Purchases": [total_purchases],
    "Total Revenue": [format_currency(total_revenue)]
})

df_purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed

In [6]:
purchase_data_grouped_sn = purchase_data.groupby('Gender')['SN'].nunique()
print(purchase_data_grouped_sn)

Gender
Female                    81
Male                     484
Other / Non-Disclosed     11
Name: SN, dtype: int64


In [7]:
gender_df = purchase_data_grouped_sn.to_frame()

In [8]:
gender_percentage = gender_df["SN"]/total_players
gender_df["Percentage of Players"] = gender_percentage.apply(lambda val: "{0:.2f}".format(val * 100))
renamed_df = gender_df.rename(columns={"SN":"Total Count"}).sort_values("Total Count", ascending=False)

renamed_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
# Purchase count by gender
purchase_data_grouped_purchaseid = purchase_data.groupby('Gender')['Purchase ID'].nunique()
gender_purchases_df = purchase_data_grouped_purchaseid.to_frame()

# Average purchase price by gender
purchase_data_grouped_price = purchase_data.groupby('Gender')['Price'].mean()
gender_avg_price_df = purchase_data_grouped_price.to_frame()


# Total Purchase Value
total_purchase_grouped = purchase_data.groupby(['Gender'])['Price'].sum()
gender_total_purchase_df = total_purchase_grouped .to_frame()


# Average purchase total per person by geneder 
purchase_avg_total_grouped = purchase_data.groupby(['Gender','SN'])['Price'].sum()
gender_avg_total_purchase_df = purchase_avg_total_grouped.to_frame()
average_purchase = gender_avg_total_purchase_df.groupby(level=0).mean() 



# Merge all dataframes - two at a time
merged_gender_df1 = pd.merge(gender_purchases_df,gender_avg_price_df, on="Gender")
merged_gender_df2 = pd.merge(merged_gender_df1, gender_total_purchase_df, on="Gender")
merged_gender_df3 = pd.merge(merged_gender_df2,average_purchase, on="Gender")

# Rename columns

merged_gender_df3 = merged_gender_df3.rename(columns={"Purchase ID":"Purchase Count", "Price_x":"Average Purchase Price",
                                              "Price_y":"Total Purchase Value","Price":"Avg Total Purchase per Person"
    
    })

# Format    
merged_gender_df3["Average Purchase Price"] = merged_gender_df3["Average Purchase Price"].astype(float).map("${:,.2f}".format)
merged_gender_df3["Total Purchase Value"] = merged_gender_df3["Total Purchase Value"].astype(float).map("${:,.2f}".format)
merged_gender_df3["Avg Total Purchase per Person"] = merged_gender_df3["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)

merged_gender_df3

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table

In [10]:
#look-up age brackets
#min_age =purchase_data["Age"].min()
#max_age =purchase_data["Age"].max()

In [11]:
bins =[0,9,14,19,24,29,34,39,50]
group_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
#pd.cut(purchase_data["Age"], bins, labels=group_labels).head()

# create a new column "Age Group"
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)
purchase_data.head()

# groupby "Age Group" and "SN" to get unique players by age group
age_group_counts= purchase_data.groupby("Age Group")['SN'].nunique()

# convert to dataframe
age_group_df = age_group_counts.to_frame()


# calculate percentages
age_percentage = age_group_df["SN"]/total_players

# add a new column
age_group_df["Percentage of Players"] = age_percentage.apply(lambda val: "{0:.2f}".format(val * 100))

# rename the SN column to Total Count
renamed_age_group_df = age_group_df.rename(columns={"SN":"Total Count"})

renamed_age_group_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, total purchase value, avg. purchase total per person by age group etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame



In [12]:
# Purchase count by age group
purchase_data_grouped_purhcaseid_age = purchase_data.groupby('Age Group')['Purchase ID'].nunique()
age_purchases_df = purchase_data_grouped_purhcaseid_age.to_frame()

# Average purchase price by age group
purchase_data_grouped_price_age = purchase_data.groupby('Age Group')['Price'].mean()
age_avg_price_df = purchase_data_grouped_price_age.to_frame()

# Total Purchase Value
total_purchase_grouped_age = purchase_data.groupby(['Age Group'])['Price'].sum()
age_total_purchase_df = total_purchase_grouped_age.to_frame()

#Average purchase total per person by geneder 
purchase_avg_total_grouped_age = purchase_data.groupby(['Age Group','SN'])['Price'].sum()
age_avg_total_purchase_df = purchase_avg_total_grouped_age.to_frame()
average_purchase_age = age_avg_total_purchase_df.groupby(level=0).mean() 

#Merge all dataframes - two at a time
merged_age_df1 = pd.merge(age_purchases_df,age_avg_price_df, on="Age Group")
merged_age_df2 = pd.merge(merged_age_df1, age_total_purchase_df, on="Age Group")
merged_age_df3 = pd.merge(merged_age_df2,average_purchase_age, on="Age Group")

# rename columns
merged_age_df3 = merged_age_df3.rename(columns={
    "Purchase ID":"Purchase Count",
    "Price_x":"Average Purchase Price",
    "Price_y":"Total Purchase Value",
    "Price":"Avg Total Purchase per Person"
})

#format    
merged_age_df3["Average Purchase Price"] = merged_age_df3["Average Purchase Price"].astype(float).map("${:,.2f}".format)
merged_age_df3["Total Purchase Value"] = merged_age_df3["Total Purchase Value"].astype(float).map("${:,.2f}".format)
merged_age_df3["Avg Total Purchase per Person"] = merged_age_df3["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)

merged_age_df3

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame


In [13]:
# Purchase Count, Averagae Purchase Price, Total Purchase Value 

#group by SN
top_spenders_grouped_sn = purchase_data.groupby('SN')
#top_spenders_grouped_sn.head()
purchase_count = top_spenders_grouped_sn["Purchase ID"].nunique()
average_purchase_price = top_spenders_grouped_sn["Price"].mean()
total_purchase_value = top_spenders_grouped_sn["Price"].sum()

df_top_spenders = pd.DataFrame({
    "Purchase Count": purchase_count,
    "Average Purchase Price": average_purchase_price,
    "Total Purchase Value": total_purchase_value
})


df_top_spenders = df_top_spenders.sort_values("Total Purchase Value", ascending=False)


df_top_spenders["Average Purchase Price"] = df_top_spenders["Average Purchase Price"].astype(float).map("${:,.2f}".format)
df_top_spenders["Total Purchase Value"] = df_top_spenders["Total Purchase Value"].astype(float).map("${:,.2f}".format)

df_top_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame

In [14]:
popular_items_grouped = purchase_data.groupby(['Item ID','Item Name'])

purchase_count_item = popular_items_grouped["Purchase ID"].nunique()
item_price = popular_items_grouped["Price"].first()
total_purchase_value_item = popular_items_grouped["Price"].sum()

df_popular_items = pd.DataFrame({
    "Purchase Count": purchase_count_item,
    "Item Price": item_price,
    "Total Purchase Value": total_purchase_value_item
})

df_popular_items = df_popular_items.sort_values("Purchase Count", ascending=False)

# creating this copy to use for most profitable items - can't sort otherwise 
df_popular_items_unformatted = df_popular_items.copy()

# formattig
df_popular_items["Item Price"] = df_popular_items["Item Price"].astype(float).map("${:,.2f}".format)
df_popular_items["Total Purchase Value"] = df_popular_items["Total Purchase Value"].astype(float).map("${:,.2f}".format)

df_popular_items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

In [15]:
df_profitable = df_popular_items_unformatted.sort_values("Total Purchase Value",ascending=False)

# formattig
df_profitable["Item Price"] = df_profitable["Item Price"].astype(float).map("${:,.2f}".format)
df_profitable["Total Purchase Value"] = df_profitable["Total Purchase Value"].astype(float).map("${:,.2f}".format)
df_profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
